In [84]:
# To obtain the API requests
import requests
# For data cleaning and display
import pandas as pd
# For file management and saving of API Tokens
import os
# To access the JSON file that contains the tweets
import json
# For parsing the time in readable format
import datetime
import dateutil.parser
# To record the time in which the tweets were obtained.
import time
import unicodedata
# For CSV format files
import csv
from dotenv import load_dotenv, find_dotenv

In [85]:
# Loading the environmental variable file to parse the API Tokens
load_dotenv(find_dotenv('Tokens.env'))

Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5


True

In [86]:
# Printing the API Bearer Token
print(os.getenv('BEARER_TOKEN'))

AAAAAAAAAAAAAAAAAAAAAJCPhwEAAAAA2uzB7HjX1IwkUEp6dPnfMj0uQUA%3D7qPei4BqVixg2LxL0gaubAVYuly7prsSijGidSZ9P9KkV8fut3


In [87]:
# Creating headers that will be used to access the APIs
def headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    return headers

In [88]:
# Creating the URL request
def CreateURL(keyword, start_date, end_date, max_results = 100):

    # Returns the tweets in JSON format
    Search_URL = "https://api.twitter.com/1.1/search/tweets.json"

    query_params = {
        'query': "theft, wezi, wizi, dungwa, visu, killed, robbed",
        'lang': "en, sw",
        'result_type' : "mixed",
        'count' : max_results,
        'start_date' : "2022-11-06",
        'end_date' : "2022-11-18",
        'place_fields' : 'Kenya',
        'next_token' : {}
    }

    return(Search_URL, query_params)

In [89]:
# Connecting the header, URL and parameters to the endpoint
def connect_to_endpoint(url, header, param, Next_Token = None):
    param['next_token'] = 'next_token'
    responses = requests.request('GET', url, headers = header, params = param)
    print("Endpoint Response Code: " +str(responses.status_code))
    if responses.status_code != 200:
        raise Exception (responses.status_code, responses.text)

    return responses.json
# Parsing the responses in JSON readable format
url = CreateURL('keyword', 'start_date', 'end_date', max_results = 100)
json_response = connect_to_endpoint(url[0], headers(bearer_token='BEARER_TOKEN'), url[1])
print(json.dumps(json_response, indent=4, sort_keys=True))
json_response['data'][0]['created_at']
json_response['meta']['result_count']

# Saving the responses as a CSV file
df = pd.DataFrame(responses['json_response'])
df.to_csv('data.csv')

Endpoint Response Code: 401


Exception: (401, '{"errors":[{"code":89,"message":"Invalid or expired token."}]}')

In [ ]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:

        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']

        # Assemble all data in a list
        res = [created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]

        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

append_to_csv(json_response, "data.csv")